# Real world Data

## Data source 

Basically, the dataset tree is the following:

- Region
    - Country
        - Industry
            - Client 
                - Products
                
## Variables name

|  Variable name        |  Type   |
| --------------------- | ------- |
| ID                    |  object |
| Country_name          | object  |
| country_ref           | object  |
| Region                | object  |
| industry              | object  |
| A_reference           | float64 |
| A_market              | float64 |
| B_reference           | float64 |
| B_market              | float64 |
| C_reference           | float64 |
| C_market              | float64 |
| D_reference           | float64 |
| D_market              | float64 |
| E_reference           | float64 |
| E_market              | float64 |
| Total_reference       | float64 |
| Total_market          | float64 |

Note that, the market includes the reference

In [ ]:
import pandas as pd
#import plotly.express as px
import numpy as np

import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)

In [ ]:
df_final = pd.read_csv('dataPandasClass.gz')
df_final.tail()

## Objectives

In this notebook, you need to:

- Print the country name by region
- Compute the market size for the reference by:
    - Product
    - Industry
    - Region
- Industry Market size by product
- Region Market size by product
- Region Market size by industry

For all the above computation compute the market share of `reference`

- Remove Region-industry with no revenue
- Compute Euclidean Distance by Region-industry-product
- Filter the data by Region-industry-product

**the market share :Revenue to potential**

$$\text{Market share} = \sum  \text{Revenue Reference} / \sum \text{Total Market} $$

## Parameters

In [ ]:
reference = [
    "A_reference",
    "B_reference",
    "C_reference",
    "D_reference",
    "E_reference",
]
market = [
    "A_market",
    "B_market",
    "C_market",
    "D_market",
    "E_market",
]

In [ ]:
df_final[reference].head()

## List country by region

In [ ]:
for c in df_final['Region'].sort_values().unique():

    countries = df_final[df_final['Region'].isin(
        [c])]['Country_name'].unique()

    print('\n Region name {} \n {}'.format(c, countries))

## Index

In [ ]:
list(df_final)

In [ ]:
df_index = df_final.set_index(['ID',
                    'Country_name',
                    'Region',
                    'industry'])
df_index.head()

In [ ]:
df_index.loc[('Customer 25')]

In [ ]:
df_index.loc[('Customer 25','Belgium')]

In [ ]:
df_index.loc[('Customer 25',
              slice(None),
              slice(None), 'A')].head()

In [ ]:
df_index.xs('Belgium',
            level='Country_name',
            axis=0, drop_level=False).head()

In [ ]:
df_index.xs(('Belgium'),
            level=('Country_name'),
            axis=0, drop_level=False).head()

In [ ]:
df_index.loc[('Customer 25'), 'A_reference']

# Top level

## Product Market size

In [ ]:
(df_final[reference + market]
 .sum(axis = 0)
 .reset_index(name = 'sum_product')
 .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
         product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
 .drop(columns = 'index')
 .sort_values(by = "product")
 .set_index(['product', 'origin'])
 .unstack(level = 1)
 .assign(market_size = lambda x: x.iloc[:, 1] / x.iloc[:, 0])
 .sort_values(by = 'market_size')
 .style
 .bar(subset=['market_size'], color='#d65f5f')
 .format("{:.2%}", subset=['market_size'])
)

## Industry Market size

In [ ]:
(df_final
 .groupby('industry').agg(
     {
         'Total_reference': 'sum',
         'Total_market': 'sum'})
 .assign(revenue_to_potential = lambda x:
         x['Total_reference']/ x['Total_market'])
 .sort_values(by = 'revenue_to_potential')
 .style
 .bar(subset=['revenue_to_potential'], color='#d65f5f')
 .format("{:.2%}", subset=['revenue_to_potential'])
          )

## Region Market size

In [ ]:
(df_final
 .groupby('Region').agg(
     {
         'Total_reference': 'sum',
         'Total_market': 'sum'})
 .assign(revenue_to_potential = lambda x: x['Total_reference']/ x['Total_market'])
 .sort_values(by = 'revenue_to_potential')
 .style
 .format({'revenue_to_potential': "{:.2%}"})
 .bar(subset=['revenue_to_potential'], color='#d65f5f')
 )

# Next level

## Industry Market size by product

In [ ]:
(df_final[['industry'] + reference + market]
 .groupby('industry')
 .sum(axis = 0)
 .transpose()
 .reset_index()
 .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
        product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
 .drop(columns = 'index')
 .sort_values(by = "product")
 .set_index(['product', 'origin'])
 .groupby(level = 0)
 .apply(lambda x: x.shift(1) / x )
 .dropna()
 .droplevel('origin')
 .transpose()
 .assign(total_industry = lambda x: x.sum(1))
 .sort_values(by ='total_industry')
 .drop(columns = 'total_industry')
 .style
 .format("{:.2%}")
 .background_gradient(cmap=cm)
)

## Region Market size by industry

In [ ]:
ttpotential_i = (df_final
 .groupby('industry').agg(
     {
         'Total_market': 'sum'})
            )
ttpotential_i.head()

In [ ]:
revenue_ci = (df_final
 .groupby(['Region', 'industry']).agg(
     {
         'Total_reference': 'sum'}
 )
)

In [ ]:
region = df_final['Region'].unique()

In [ ]:
(
    revenue_ci.reset_index()
    .merge(ttpotential_i.reset_index(), on="industry", how="inner")
    .assign(
        revenue_to_potential=lambda x: x["Total_reference"] / x["Total_market"]
    )
    .sort_values(by=["Region", "revenue_to_potential"], ascending=[True, False])
    .drop(columns=["Total_reference", "Total_market"])
    .pivot(index="industry", columns="Region", values="revenue_to_potential")
    .assign(total_industry = lambda x: x.sum(1))
    .sort_values(by ='total_industry')
    .fillna(0)
    .style
    .highlight_null('red')
    .background_gradient(cmap=cm,
                      subset = region)
    .bar(subset=['total_industry'], color='#d65f5f')
)

## Correlation

In [ ]:
df_final.corr()

## Euclidean Distance

## Definition Euclidean Distance

According to the Euclidean distance formula, the distance between two points in the plane with coordinates (x, y), and is given by:

$$\begin{aligned} d(\mathbf{p}, \mathbf{q})=d(\mathbf{q}, \mathbf{p}) &=\sqrt{\left(q_{1}-p_{1}\right)^{2}+\left(q_{2}-p_{2}\right)^{2}+\cdots+\left(q_{n}-p_{n}\right)^{2}} \\ &=\sqrt{\sum_{i=1}^{n}\left(q_{i}-p_{i}\right)^{2}} \end{aligned}$$

### Example 

Calculate the distance between two arrays.

We want to sum each column of the first array with all the columns of the second array

Take an example, we want to compute the Euclidean distance of this first array:
- [7.2, 3.6, 5.1, 2.5]: It has 4 columns and one row

we will compute the Euclidean distance with all the rows of the second array. The second array has also four columns. We are substracting each columns of the first and second array respecively 

Exemple with the first rows of the second array:
- [5.1, 3.5, 1.4, 0.2]

The Euclidean distance is $$\sqrt{((7.2 - 5.1)^2  + (3.6 - 3.5)^2 + (5.1 -1.4)^2 + (2.5 - 0.2)^2)}$$

We repeat for all the rows

## Data standardization

The value of distance measures is intimately related to the scale on which measurements are made.

In [ ]:
def euclideanDistance(vect_1, vect_2):
  
    """
      Compute the Euclidean distance 
    """
  
    sum_ = np.sum(np.power(vect_1 - vect_2, 2))
  
    euclideanD = np.sqrt(sum_)
    return euclideanD

Remove country industry pair with no remove.It will inflate the distance

In [ ]:
indu_norevenue = (df_final
 .groupby(['Region', 'industry'])
 .agg(
     {
         'Total_reference': 'sum'}
 )
 .loc[lambda x: x['Total_reference'] == 0]
)
indu_norevenue

In [ ]:
list_ = list(indu_norevenue.index)
list_[0][0]

In [ ]:
index_no = [] 
for c in  list_:
    index_ =  df_final[(df_final['Region'] == c[0]) &
          (df_final['industry'] == c[1])
          ].index
    index_no.extend(index_)

In [ ]:
X = df_final[~df_final.index.isin(index_no)]

In [ ]:
def compute_potential(df, normalized=True, begin=0, end=10):
    """
    The fonction compute the Euclidean Distance
    Merge the whitespace 
    and plot the results: selected points and whitespace
    """

    # Normal

    # Reshape the data
    region_indu_prod = (df[['Region', 'industry'] + reference + market]
                        .groupby(['Region', 'industry'])
                        .sum(axis=0)
                        .stack()
                        .reset_index(name='sum')
                        .assign(origin=lambda x: x['level_2'].str.extract(
                            r"(_[^_]+$)"),
                                product=lambda x: x['level_2'].str.extract(
                                    r"(^[^_]+(?=_))"))
                        .drop(columns='level_2')
                        .set_index(['Region', 'industry', 'product', 'origin'
                                    ])
                        .unstack()
                        .assign(norm_pot=lambda x:  (x.iloc[:, 0] -
                                                     x.iloc[:, 0].mean())/
                                x.iloc[:, 0].std(),
                                norm_vod=lambda x:  (
                                x.iloc[:, 1] - x.iloc[:, 1].mean())/
                                x.iloc[:, 1].std(),
                                )
                        )

    # Create distance
    if normalized:
        x_ = 2
        y_ = 3
    else:
        x_ = 0
        y_ = 1

    region_indu_prod["distance"] = region_indu_prod.groupby(
        #['Inbound_Region', 'str']
        level=[0, 1]
    ).apply(lambda x: euclideanDistance(x.iloc[:, x_],
                                        x.iloc[:, y_],
                                        )
            )

    ### Compute whitesaoce
    whitespace_ri = (
        region_indu_prod
        .iloc[:, :2]
        .droplevel('product')
        .groupby(level=[0, 1])
        .sum()
        .assign(whitespace=lambda x: x.iloc[:, 0] - x.iloc[:, 1])
        .iloc[:, -1]
    )

    sort_distance = (region_indu_prod
                     .droplevel(level=[2])
                     .iloc[:, -1]
                     .drop_duplicates()
                     )
    df_Epotential = pd.concat(
        [sort_distance, whitespace_ri], axis=1).sort_values(by='distance')
    
    title = 'Top {} markets: Whitespace according to \
    Euclidean Distance'.format(end - begin)

    df_Epotential.iloc[begin:end, 1].plot.barh(
        title=title)

    return region_indu_prod, df_Epotential

In [ ]:
def filter_distance(df, country, industry):
    """
    filter the multi-index dataframe
    and return country industry
    """
    return df.loc[(country,industry)]

In [ ]:
df_Epotential = compute_potential(
    df = X,
    normalized =  False,
                  begin = 80,
                  end = 92)

In [ ]:
filter_distance(df =df_Epotential[0],
                country = 'Americas',
                industry = 'E')